#０．初期処理

In [ ]:
## ライブラリインポート
from google.colab import drive
from bs4 import BeautifulSoup
import pandas as pd
import os
import glob
import datetime
import pytz
import shutil

ROOT_FOLDER_PATH = '/content/drive/MyDrive/01.プログラミング/scrape'
ROOT_FINISH_HTML_FOLDER_PATH = f"{ROOT_FOLDER_PATH}/処理完了"

today_finish_folder_name = datetime.datetime.now(pytz.timezone("Asia/Tokyo")).strftime("%Y%m%d")
today_finish_folder_path = f"{ROOT_FINISH_HTML_FOLDER_PATH}/{today_finish_folder_name}"

In [ ]:
# ドライブマウント
drive.mount('/content/drive')

Mounted at /content/drive


## ０－１．関数

In [ ]:
def read_file(read_file_path: str) -> str:
  """
  引数のファイルを読み込み、そのファイルの文字列を返す関数
  """
  file_str = ""
  with open(read_file_path, "r") as f:
    file_str = f.read()

  return file_str

In [ ]:
def scrape_html(read_html_name: str, file_index: int, read_html_doc: str) -> dict:
  """
  引数のHTMLテキストから特定の属性値を抽出し、dict型に変換する関数
  """
  soup = BeautifulSoup(read_html_doc, 'html.parser')
  tags = soup.find_all("a")

  anchor_tag_dicts = []

  for tag in tags:
    print(str(tag))
    print(type(tag.string))
    anchor_tag_dicts.append({
        "read_html": os.path.basename(read_html_name),
        "file_index": file_index,
        "href": tag.get("href"),
        "ga_link": tag.get("ga_link"),
        "data_ga_event_category": tag.get("data-event_category"),
        "data_ua_event": tag.get("data-ua-event")
    })

  return anchor_tag_dicts

# １．ファイル→BigQueryへロード

In [ ]:
# 対象ファイル読み込み
target_html_file_paths = glob.glob(f"{ROOT_FOLDER_PATH}/*.html")

os.makedirs(today_finish_folder_path, exist_ok=True)
anchor_tag_dicts = []

for file_index in range(len(target_html_file_paths)):
  read_html_path = target_html_file_paths[file_index]
  try :
    # html読込
    read_html_doc = read_file(read_html_path)

    # HTMLから特定の要素をスクレイプ
    anchor_tag_dicts.extend(scrape_html(
        os.path.basename(read_html_path),
        file_index,
        read_html_doc
    ))

    # 読み込み後のHTMLは完了フォルダに移動
    # shutil.move(read_html_path, today_finish_folder_path)
  except Exception as e:
    pass

In [ ]:
anchor_tag_df = pd.DataFrame(anchor_tag_dicts)